In [10]:
import os

# Get and print the current working directory
current_working_directory = os.getcwd()
print(f"The current working directory is: {current_working_directory}")

The current working directory is: /content/sample_data/Flappy_birds/src


In [18]:
import os

# Create directories if they don't exist
if not os.path.exists("/content/sample_data/Flappy_birds/src/tensorboard"):
    os.makedirs("/content/sample_data/Flappy_birds/src/tensorboard")

if not os.path.exists("/content/sample_data/Flappy_birds/src/"):
    os.makedirs("/content/sample_data/Flappy_birds/src/")

# Change the working directory to a new directory (replace with the path you want)
new_working_directory = "/content/sample_data/Flappy_birds/src/"
os.chdir(new_working_directory)



In [ ]:
def get_args():
    parser = argparse.ArgumentParser(
        """Implementation of Deep Q Network to play Flappy Bird""")
    parser.add_argument("--image_size", type=int, default=84, help="The common width and height for all images")
    parser.add_argument("--saved_path", type=str, default="trained_models")

    args = parser.parse_args()
    return args


def test(opt):
    if torch.cuda.is_available():
        torch.cuda.manual_seed(123)
    else:
        torch.manual_seed(123)
    if torch.cuda.is_available():
        model = torch.load("{}/flappy_bird".format(opt.saved_path))
    else:
        model = torch.load("{}/flappy_bird".format(opt.saved_path), map_location=lambda storage, loc: storage)
    model.eval()
    game_state = FlappyBird()
    image, reward, terminal = game_state.next_frame(0)
    image = pre_processing(image[:game_state.screen_width, :int(game_state.base_y)], opt.image_size, opt.image_size)
    image = torch.from_numpy(image)
    if torch.cuda.is_available():
        model.cuda()
        image = image.cuda()
    state = torch.cat(tuple(image for _ in range(4)))[None, :, :, :]

    while True:
        prediction = model(state)[0]
        action = torch.argmax(prediction) #[0]

        next_image, reward, terminal = game_state.next_frame(action)
        next_image = pre_processing(next_image[:game_state.screen_width, :int(game_state.base_y)], opt.image_size,
                                    opt.image_size)
        next_image = torch.from_numpy(next_image)
        if torch.cuda.is_available():
            next_image = next_image.cuda()
        next_state = torch.cat((state[0, 1:, :, :], next_image))[None, :, :, :]

        state = next_state


if __name__ == "__main__":
    from argparse import Namespace

    # Set the arguments here instead of using command-line arguments
    args = Namespace(image_size=84, saved_path="trained_models")

    test(args)
